In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import os

import numpy as np

from torch_model import TextRNN, TextCNN
from cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab

2023-12-12 14:48:54.806222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 14:48:54.963564: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 14:48:54.967141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr

In [2]:
base_dir = 'cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def evaluate(model, Loss, x_val, y_val):
    """测试集上准确率评估"""
    batch_val = batch_iter(x_val, y_val, 64)
    acc = 0
    los = 0
    for x_batch, y_batch in batch_val:
        size = len(x_batch)
        x = np.array(x_batch)
        y = np.array(y_batch)
        x = torch.LongTensor(x).to(device)
        y = torch.Tensor(y).to(device)
        # y = torch.LongTensor(y)
        # x = Variable(x)
        # y = Variable(y)
        out = model(x)
        loss = Loss(out, y)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        loss_value = np.mean(loss.cpu().detach().numpy())
        accracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        acc +=accracy*size
        los +=loss_value*size
    return los/len(x_val), acc/len(x_val)

In [4]:
def train():
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id,600)#获取训练数据每个字的id和对应标签的oe-hot形式
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id,600)
    #使用LSTM或者CNN
    # model = TextRNN()
    model = TextCNN()
    
    model.to(device)
    
    #选择损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # Loss = nn.BCELoss()
    # Loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    best_val_acc = 0
    for epoch in range(20):
        i = 0
        print('epoch:{}'.format(epoch))
        batch_train = batch_iter(x_train, y_train,64)
        for x_batch, y_batch in batch_train:
            i +=1
            # print(i)
            x = np.array(x_batch)
            y = np.array(y_batch)
            x = torch.LongTensor(x).to(device)
            y = torch.Tensor(y).to(device)
            # y = torch.LongTensor(y)
            # x = Variable(x)
            # y = Variable(y)
            out = model(x)
            loss = Loss(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 对模型进行验证
            if i % 300 == 0:
                los, accracy = evaluate(model, Loss, x_val, y_val)
                print('loss:{},accracy:{}'.format(los, accracy))
                if accracy > best_val_acc:
                    torch.save(model.state_dict(), 'model_params_cnn.pkl')
                    best_val_acc = accracy

In [5]:
#获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
#获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab(vocab_dir)
#获取字数
vocab_size = len(words)
print('train')
train()

train
epoch:0
loss:0.2626219564437866,accracy:0.4482
loss:0.21302579765319823,accracy:0.5826
epoch:1
loss:0.16581347000598906,accracy:0.69
loss:0.13692533996105194,accracy:0.763
epoch:2
loss:0.1138684547662735,accracy:0.8132
loss:0.1092594182908535,accracy:0.8184
epoch:3
loss:0.10215294127464294,accracy:0.837
loss:0.09696827182769775,accracy:0.8396
epoch:4
loss:0.09490450357198715,accracy:0.8474
loss:0.09163130110949277,accracy:0.856
epoch:5
loss:0.08755503225326539,accracy:0.8622
loss:0.08077967715263366,accracy:0.8782
epoch:6
loss:0.08257041132450103,accracy:0.8724
loss:0.08029311288595199,accracy:0.8818
epoch:7
loss:0.08106081169843674,accracy:0.8852
loss:0.08329814631044864,accracy:0.8774
epoch:8
loss:0.07828249506950379,accracy:0.885
loss:0.07896101083755493,accracy:0.8838
epoch:9
loss:0.0765984297990799,accracy:0.889
loss:0.08004815601110458,accracy:0.8802
epoch:10
loss:0.07919044415950775,accracy:0.8874
loss:0.07773881826400757,accracy:0.892
epoch:11
loss:0.07355061296224594,acc